In [ ]:
from transformers import Trainer, TrainingArguments

from transformers import AutoModelForSequenceClassification, AutoTokenizer

from datasets import Dataset
import torch

---

In [ ]:
pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `colab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab`


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("ProtectAI/deberta-v3-base-prompt-injection-v2")
model = AutoModelForSequenceClassification.from_pretrained("ProtectAI/deberta-v3-base-prompt-injection-v2", num_labels=2)


def make_contiguous(model):
    for name, param in model.named_parameters():
        if not param.is_contiguous():
            param.data = param.data.contiguous()

make_contiguous(model)

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/994 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [ ]:
# define metrics

import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    predictions, labels = p
    # Convert predictions to tensor if they are a numpy array
    predictions = torch.tensor(predictions)

    # Get the predicted class labels (argmax returns the index of the maximum value along the specified axis)
    preds = torch.argmax(predictions, axis=1)

    # Calculate precision, recall, f1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
import pandas as pd

path_to_df_train='40k__train_sampled_with_embeddings.parquet'
path_to_df_test='40k__test_sampled_with_embeddings.parquet'

df_train_sampled = pd.read_parquet(path_to_df_train)
df_test_sampled = pd.read_parquet(path_to_df_test)

In [ ]:
train_encodings = tokenizer(df_train_sampled['text'].to_list(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(df_test_sampled['text'].to_list(), truncation=True, padding=True, max_length=512)

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_encodings)
test_dataset = Dataset.from_dict(test_encodings)

In [ ]:
train_dataset = train_dataset.add_column('label', df_train_sampled['label'].values)
test_dataset = test_dataset.add_column('label', df_test_sampled['label'].values)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory for model checkpoints
    num_train_epochs=20,              # number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    evaluation_strategy="epoch",     # evaluate after each epoch
    save_strategy="epoch",           # save after each epoch
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,                # how often to log
    load_best_model_at_end=True,     # load the best model at the end of training
    metric_for_best_model="recall",     # the metric to monitor for selecting the best model
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,                         # the model to train
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics=compute_metrics      # evaluation metrics
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.539600,0.517857,0.744750,0.701856,0.851000,0.769266
2,0.547400,0.501963,0.751250,0.709114,0.852000,0.774018
3,0.558200,0.492578,0.778500,0.758709,0.816750,0.786660
4,0.426400,0.470795,0.773125,0.744571,0.831500,0.785638
5,0.342600,0.505350,0.777750,0.790838,0.755250,0.772634
6,0.322000,0.539464,0.782875,0.757921,0.831250,0.792894
7,0.369900,0.601766,0.778500,0.731120,0.881000,0.799093
8,0.390100,0.717209,0.781875,0.771883,0.800250,0.785811
9,0.316100,0.671512,0.782250,0.781266,0.784000,0.782630
10,0.221000,0.793829,0.783125,0.776422,0.795250,0.785723


TrainOutput(global_step=40000, training_loss=0.26681553905531763, metrics={'train_runtime': 57774.6731, 'train_samples_per_second': 11.078, 'train_steps_per_second': 0.692, 'total_flos': 1.6839409532928e+17, 'train_loss': 0.26681553905531763, 'epoch': 20.0})

In [ ]:
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.6017656326293945, 'eval_accuracy': 0.7785, 'eval_precision': 0.7311203319502074, 'eval_recall': 0.881, 'eval_f1': 0.7990929705215419, 'eval_runtime': 237.8327, 'eval_samples_per_second': 33.637, 'eval_steps_per_second': 0.526, 'epoch': 20.0}


In [ ]:
trainer.save_model('/content/drive/MyDrive/deberta-v3-base-prompt-injection-v2_model_20_epochs')